### Подготовка данных.

Libraries

In [106]:
# for data
import os
import torch
import glob

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim

from PIL import Image
from sklearn.model_selection import train_test_split

# for training
import torch.nn as nn

Static configs

In [107]:
PATH_2_SAMPELS = 'samples/*'
TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'

BATCH_SIZE = 8

ALPHABETH = 'abcdefghijklmnopqrstuvwxyz'
DIGITS = '0123456789'
CHARS = ALPHABETH + DIGITS
VOCAB_SIZE = len(CHARS) + 1

EPOCHS = 10

In [140]:
class CaptchDataset(Dataset):
    """
    class for getting data and labels
    """
    def __init__(self, pth):
        """
        collect data
        """
        pth_list = os.listdir(pth)
        abs_pth = os.path.abspath(pth)
        print(abs_pth)
        self.imgs = [os.path.join(abs_pth, p) for p in pth_list]
        self.transform = transforms.Compose([
          transforms.ToTensor()  
        ])
    
    def __len__(self):
        """
        get length of the data
        """
        return len(self.imgs)
    
    def __getitem__(seld, idx):
        """
        get tensor of the image and label by index
        """
        pth = self.img_list[idx]
        label = os.path.basename(pth).split('.')[0].lower().strp()
        img = Image.open(pth).convert('RGB')
        img_tensor = self.transform(img)
        return img_tensor, label
      

In [141]:
train_dataset.__dict__

{'imgs': [],
 'transform': Compose(
     ToTensor()
 )}

In [147]:
# prepare folders
os.system(f'mkdir data')
os.system(f"rm -rf data/train")
os.system(f"rm -rf data/test")
[os.system(f'mkdir data/{directory}') for directory in ('train', 'test')]


# split data for train and test 1/4
os.system(f'mkdir data')
os.system(f"rm -rf data/train")
os.system(f"rm -rf data/test")
[os.system(f'mkdir data/{directory}') for directory in ('train', 'test')]

[os.system(f"cp {te} {VAL_DIR}/{te.split('/')[1]}") for te in test_set]
[os.system(f"cp {tr} {TRAIN_DIR}/{tr.split('/')[1]}") for tr in train_set]

train_dataset = CaptchDataset(TRAIN_DIR)
val_dataset = CaptchDataset(VAL_DIR)


# loading data
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE,
    shuffle=False
)


mkdir: data: File exists
mkdir: data: File exists


### Создание и обучение модели.

CRNN class with layers

In [165]:
class CNNRNN(nn.Module):
    def __init__(self, vocab_size, dropout=0.5):
        """
        initiate layers
        """
        super(CNNRNN, self).__init__()
        # probability of an element to be zeroed (Bernoulli distribution)
        self.dropout = nn.Dropout(p=dropout)
        self.clayer = nn.Sequential(
            # With square kernels and equal stride and with padding
            nn.Conv2d(
                in_channels=3,
                out_channels=32, 
                kernel_size=(3,3),
                stride=1,
                padding=1
            ),
            # activation max(0,x)
            nn.ReLU(), 
            nn.MaxPool2d(
                kernel_size=(2,2),
                stride=2
            ),
            # in_channels, out_channels, kernel_size
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=(3,3),
                stride=1, 
                padding=1),
            nn.ReLU(),
            # pool of square window of size=2, stride=2
            nn.MaxPool2d(
                kernel_size=(2,2), 
                stride=2
            ),
            
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=(3,3), 
                stride=1, 
                padding=1
            ),
            nn.ReLU(), # max(0,x)
            
            nn.Conv2d(
                in_channels=128,
                out_channels=256, 
                kernel_size=(3,3), 
                stride=1, 
                padding=1
            ),
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=(1,2), 
                stride=2
            ),
            
            nn.Conv2d(
                in_channels=256,
                out_channels=512,
                kernel_size=(3,3), 
                stride=1, 
                padding=1
            ),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            
            nn.Conv2d(
                in_channels=512,
                out_channels=512,
                kernel_size=(3,3), 
                stride=1, 
                padding=1
            ),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=(1, 2), 
                stride=2),
            
            nn.Conv2d(
                in_channels=512,
                out_channels=512,
                kernel_size=(2,2), 
                stride=1, 
                padding=1
            ),
            self.dropout
        )
        self.seq = nn.Sequential(
            nn.Linear(
                in_features=1024, 
                out_features=256
            ),
            self.dropout
        )
        # RNN to an input sequence
        self.rnn_layer = nn.GRU(
            input_size=256,
            hidden_size=256, 
            num_layers=2, 
            bidirectional=True
        )
        # Linear transformation to the incoming data
        self.outlayer = nn.Linear(
            in_features=512, 
            out_features=vocab_size
        )

        
    def forward(self, x):
        """
        method for CNN and RNN combining 
        """
        x = self.clayer(x) # CNN apply
        x = x.permute(0, 3, 1, 2) # transformate dimension
        x = x.view(x.size(0), x.size(1), -1) # choose field
        x = self.seq(x) # apply linear with dropout
        x, _ = self.rnn_layer(x) # apply rnn layers
        x = self.outlayer(x) # apply linear layer
        return x.permute(1, 0, 2) # transformate dimension
        

In [184]:
class OCR:
    def __init__(self):
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.crnn = CNNRNN(VOCAB_SIZE).to(self.device)
        self.critertion = nn.CTCLoss(blank=0)
        self.char2idx, self.idx2char = self.char_idx()
        
        
    def char_idx(self):
        char2idx = {}
        idx2char = {}
        chars = CHARS.lower() + '-'
        for i, chara in enumerate(chars):
            char2idx[chara] = i + 1
            idx2char[i + 1] = chara
        return char2idx, idx2char
    
    def encode(self, labels):
        length_per_label = [len(label) for label in labels]
        joined_label = ''.join(labels)
        joined_encoding = []
        for chara in joined_label:
            joined_encoding.append(self.char2idx[chara])
        return torch.IntTensor(joined_encoding), torch.IntTensor(length_per_label)
    
    def decode(self, logits):
        tokens = logits.softmax(2).argmax(2).sqeeze(1)
        tokens = ''.join([self.ix2char[token]
                         if token != 0 else '-'
                         for token in tokens.numpy()])
        tokens = tokens.split('-')
        
        text = [chara for batch_token in tokens
               for idx, chara in enumerate(batch_token)
               if chara != batch_token[idx-1] or len(barth_token) == 1]
        text = ''.join(text)
        return text
    
    def loss_func(self, logits, labels):
        encoded_labels, labels_len = self.encode(labels)
        logits_lens = torch.full(
            size=(logits.size(1),),
            fill_value=logits.size(0),
            dtype=torch.int32
        ).to(self.device)
        
        return self.critertion(
            logits.log_softmax(2),
            encoded_labels,
            logits_lens,
            labels_len
        )
        
    def val_step(self, images, labels):
        logits = self.predict(images)
        loss = self.loss_func(logits, labels)
        return logits, loss
    
    def train_step(self, optimizer, images, labels):
        logits = self.predict(images)
        optimizer.zero_grad()
        loss = self.loss_func(logits, labels)
        loss.backward()
        optimizer.step()
        return logits, loss
    
    def predict(self, img):
        return self.crnn(img.to(self.device))
    
    def train(self, num_epochs, optimizer, train_loader, test_loader, print_every=2):
        train_losses, valid_losses = [], []
        for epoch in range(num_epochs):
            total_train_loss = 0
            self.crnn.train()
            for i, (images, labels) in enumerate(train_loader):
                logits, train_loss = self.train_step(
                    optimizer,
                    images,
                    labels
                )
                total_train_loss += train_loss.item()
            
            with torch.no_grad():
                total_val_loss = 0
                self.crnn.eval()
                for i, (images, labels) in enumerate(test_loader):
                    logits, val_loss = self.val_step(images, labels)
                    total_val_loss += val_loss.item()
                train_loss = total_train_loss / len(train_loader.dataset)
                val_loss = total_val_loss / len(test_loader.dataset)
                train_losses.append(train_loss)
                valid_losses.append(val_loss)
            if epoch % print_every == 0:
                print(f'Epoch {epoch} | train loss {train_loss} | valid loss {val_loss}')
        return train_losses, valid_losses
        

In [185]:
ocr = OCR()
optimizer = optim.SGD(
            ocr.crnn.parameters(), 
            lr=0.02, 
            nesterov=True,
            weight_decay=1e-5,
            momentum=0.7
        )

train_losses, val_losses = ocr.train(
    EPOCHS,
    optimizer, 
    train_loader,
    val_loader
)

Epoch 0 | train loss 0.4274300157342992 | valid loss 0.4842488698870222
Epoch 2 | train loss 0.4110152246053345 | valid loss 0.45506636673044937
Epoch 4 | train loss 0.3541468947599734 | valid loss 0.3445999867448183
Epoch 6 | train loss 0.18630591564089338 | valid loss 0.23117989803029
Epoch 8 | train loss 0.07404775445582587 | valid loss 0.0740313958898883
